In [41]:
import xml.dom.minidom as minidom
import xml.etree.cElementTree as cet
import xml.etree.ElementTree as et
import os
import re
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from PIL import Image

number_of_samples = 10000
display = False
path = "CCPD2019/ccpd_base"
new_path = "data-carplate"
images_path =  "images"
templates_path = "templates"

all_annotations = {

}
counter = 0
for obj in os.listdir(path):
    filename = os.path.join(path, obj)
    if os.path.isfile(filename):
        split_data = obj.split('-')
        image_type = "." + split_data[-1].split(".")[-1]
        coordinate_pair = re.split('&|_', split_data[2])
        tilt = split_data[1].split("_")
        if abs( 100 - int(tilt[0])) > 20 or abs(100-int(tilt[1])) > 20:
            continue
        # if (split_data[1].split('_')[0 ])
        # print(tilt)
        img_path = os.path.join(images_path, str(counter) + image_type)
        template_path = os.path.join(templates_path, str(counter) + image_type)
        image_data = {
            'image_path': img_path,
            'template_path': template_path,
            'area': split_data[0],
            'tilt': {
                'horizontal': tilt[0],
                'vertical': tilt[1]
            },
            'bbox': {
                'xmin': coordinate_pair[0],
                'ymin': coordinate_pair[1],
                'xmax': coordinate_pair[2],
                'ymax': coordinate_pair[3]
            },
            'vertices': split_data[3],
            'license_plate': split_data[4],
            'brightness': split_data[5],
            'blurriness': split_data[6].split('.')[0]
        }
        all_annotations[counter] = image_data
        # print(split_data)
        # print(image_data)
        # print(filename)
        coordinates = [int(x) for x in coordinate_pair]
        img = Image.open(filename)
        template = img.crop(coordinates)
        img.save(os.path.join(new_path, img_path))
        template.save(os.path.join(new_path, template_path))
        if display:
            fig = plt.figure(figsize=(15, 20))
            fig.add_subplot(1, 2, 1)
            # plt.axis('off')
            plt.imshow(template)
            fig.add_subplot(1, 2, 2)
            # plt.axis('off')
            plt.imshow(img)
            x1, y1, x2, y2 = coordinates

            rect = patches.Rectangle((x1,y1),x2-x1+1,y2-y1+1,linewidth=1,edgecolor='r',facecolor='none')
            plt.gca().add_patch(rect)

            plt.show()

        counter += 1
    if counter >= number_of_samples:
        break

with open(os.path.join(new_path, 'annotations.json'), 'w') as outfile:
    json.dump(all_annotations, outfile)
# print(all_annotations)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

